## 마스크 착용 판별기 (얼굴인식)

### 1. webcam을 통해 10프레임마다 이미지 저장하여 모델의 데이터셋 생성

### 2. InceptionResNetV2 통해 마스크 착용 판별기 훈련
1. InceptionResNetV2 Weight 그대로사용, 마지막 레이어만 제거
2. fine tuning
    - layer 추가
    - lr 10~0.00001
    - epochs, batch_size
    - val_loss 값의 변화로 overfitting 방지

### 3. 영상으로 결과값 출력

0. 사용된 모듈

In [ ]:
import cv2
import cvlib as cv
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

1. 데이터셋 생성

In [2]:
import cv2
import cvlib as cv
 
# open webcam (웹캠 열기)
webcam = cv2.VideoCapture(0)
 
if not webcam.isOpened():
    print("Could not open webcam")
    exit()
    
 
sample_num = 0 # 데이터 수집 시작
captured_num = 0 # 데이터 수집 시작
    
# loop through frames
while webcam.isOpened():
    
    # read frame from webcam 
    status, frame = webcam.read()
    sample_num = sample_num + 1
    
    if not status:
        break
 
    # 이미지 내 얼굴 검출
    face, confidence = cv.detect_face(frame)
    
    #print(face)
    #print(confidence)
 
    # loop through detected faces
    for idx, f in enumerate(face):
        
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]
 
 
        if sample_num % 10  == 0:
            captured_num = captured_num + 1
            face_in_img = frame[startY:endY, startX:endX, :]
            cv2.imwrite('./mask/face'+str(captured_num)+'.jpg', face_in_img) # 마스크 착용 데이터 수집시 주석해제
            #cv2.imwrite('./nomask/face'+str(captured_num)+'.jpg', face_in_img) # 마스크 미착용 데이터 수집시 주석해제
 
 
    # display output
    cv2.imshow("captured frames", frame)        
    
    # q 누르면 종료 or 데이터 캡처 완료하면 종료
    if cv2.waitKey(1) == ord('q') or captured_num == 300 : # 원하는 샘플 수 
        break
    
# release resources
webcam.release()
cv2.destroyAllWindows()   

2. 데이터 전처리

In [2]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
 
 
path_dir1 = './nomask/'
path_dir2 = './mask/'
 
file_list1 = os.listdir(path_dir1) # path에 존재하는 파일 목록 가져오기
file_list2 = os.listdir(path_dir2)
 
file_list1_num = len(file_list1)
file_list2_num = len(file_list2)
 
file_num = file_list1_num + file_list2_num
 
 
# 이미지 전처리
num = 0
all_img = np.float32(np.zeros((file_num, 224, 224, 3))) 
all_label = np.float32(np.zeros((file_num, 1)))
 
for img_name in file_list1:
    img_path = path_dir1+img_name
    img = load_img(img_path, target_size=(224, 224))
    
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    all_img[num, :, :, :] = x
    
    all_label[num] = 0 # nomask
    num = num + 1
 
for img_name in file_list2:
    img_path = path_dir2+img_name
    img = load_img(img_path, target_size=(224, 224))
    
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    all_img[num, :, :, :] = x
    
    all_label[num] = 1 # mask
    num = num + 1
 
 
# 데이터셋 섞기(적절하게 훈련되게 하기 위함) 
np.random.seed(1004)
n_elem = all_label.shape[0]
indices = np.random.choice(n_elem, size=n_elem, replace=False)
 
all_label = all_label[indices]
all_img = all_img[indices]
 
 
# 훈련셋 테스트셋 분할
num_train = int(np.round(all_label.shape[0]*0.8))
num_test = int(np.round(all_label.shape[0]*0.2))
 
train_img = all_img[0:num_train, :, :, :]
test_img = all_img[num_train:, :, :, :] 
 
train_label = all_label[0:num_train]
test_label = all_label[num_train:]

3. 학습모델

In [ ]:
import matplotlib.pyplot as plt

# create the base pre-trained model
IMG_SHAPE = (224, 224, 3)
 
base_model = InceptionResNetV2(input_shape=IMG_SHAPE, weights='imagenet', include_top=False)
base_model.trainable = False
#base_model.summary()
print("Number of layers in the base model: ", len(base_model.layers))
 
flatten_layer = Flatten()
dense_layer1 = Dense(128, activation='relu')
bn_layer1 = BatchNormalization()
dense_layer2 = Dense(1, activation=tf.nn.sigmoid)
 
model = Sequential([
        base_model,
        flatten_layer,
        dense_layer1,
        bn_layer1,
        dense_layer2,
        ])
 
#early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=5, verbose=1)

base_learning_rate = [10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001]

for l in base_learning_rate:
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=l), loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(train_img, train_label, epochs=20, batch_size=16, validation_data = (test_img, test_label))
    model.save("model_pract{}.h5".format(l))

    print("Saved model to disk")

    #결과를 그래프로 표현하는 부분입니다.
    
    acc= history.history['accuracy']
    val_acc= history.history['val_accuracy']
    y_vloss = history.history['val_loss']
    y_loss = history.history['loss']

    x_len = np.arange(len(y_loss)) 
    plt.suptitle(l)
    plt.subplot(1, 2, 1)                # nrows=1, ncols=1, index=1
    plt.plot(x_len, y_vloss, marker='.', c="cornflowerblue", label='Testset_loss')
    plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')
    plt.title('loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='upper right')

    plt.subplot(1, 2, 2)                # nrows=1, ncols=2, index=2
    plt.plot(x_len, val_acc, marker='.', c="lightcoral", label='Testset_acc')
    plt.plot(x_len, acc, marker='.', c="red", label='Trainset_acc')
    plt.title('acc')
    plt.xlabel('epoch')
    plt.ylabel('acc')
    plt.legend(loc='lower right') 
    
    plt.grid()
    plt.show()

4. 마스크 착용 판별기

In [3]:
import cvlib as cv
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
 
model = load_model('model_colab_lr_0.001.h5') # colab 딥러닝 모델 불러오기
# model.summary()
 
# open webcam
webcam = cv2.VideoCapture(0)

# 동영상 저장용 코드
w = round(webcam.get(cv2.CAP_PROP_FRAME_WIDTH))
h = round(webcam.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = webcam.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('output.avi', fourcc, fps*0.4, (w, h))
 
if not webcam.isOpened():
    print("Could not open webcam")
    exit()
    
 
# loop through frames
while webcam.isOpened():
 
    # read frame from webcam 
    status, frame = webcam.read()
    
    if not status:
        print("Could not read frame")
        exit()
 
    # apply face detection
    face, confidence = cv.detect_face(frame)
 
    # loop through detected faces
    for idx, f in enumerate(face):
        
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]
        
        if 0 <= startX <= frame.shape[1] and 0 <= endX <= frame.shape[1] and 0 <= startY <= frame.shape[0] and 0 <= endY <= frame.shape[0]:
            
            face_region = frame[startY:endY, startX:endX]
            
            face_region1 = cv2.resize(face_region, (224, 224), interpolation = cv2.INTER_AREA)
            
            x = img_to_array(face_region1)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            
            prediction = model.predict(x)
 
            if prediction < 0.5: # 마스크 미착용으로 판별되면
                cv2.rectangle(frame, (startX,startY), (endX,endY), (0,0,255), 2)
                Y = startY - 10 if startY - 10 > 10 else startY + 20
                text = "No Mask ({:.2f}%)".format((1 - prediction[0][0])*100)
                cv2.putText(frame, text, (startX,Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                
            else: # 마스크 착용으로 판별되면
                cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)
                Y = startY - 10 if startY - 10 > 10 else startY + 20
                text = "Mask ({:.2f}%)".format(prediction[0][0]*100)
                cv2.putText(frame, text, (startX,Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
                
    # display output
    cv2.imshow("mask nomask classify", frame)
    out.write(frame)
    # press "Q" to stop
    if cv2.waitKey(1) == ord('q'):
        break
    
# release resources
webcam.release()
out.release()
cv2.destroyAllWindows() 